# Using music21 to parse MIDI data

In [3]:
from music21 import *

Let's start with a piece Ave Maria...

In [ ]:
sBach = converter.parse("MIDI_Archive/Classical_Greatest/Bach/AveMaria.mid")
sBach.show() #for this to work, you have to install a software called musescore

We can also play it!

In [ ]:
sBach.show('midi')

# Extract MIDI sequence

To make a network, we want to extract the MIDI sequence for different parts. After piece is parsed, you can access different parts using piece.parts, and for each part p, p.notesAndRests would access the notes (or chords, aka notes happening together) and rests in the sequence, and secondsMap will provide the information of not only the note pitch, but also when it starts and when it ends.

In [4]:
def quantify_music(piece):
    midi_dict = {}
    time_dict = {}
    key = piece.analyze('key').name # tell us the key of the piece (eg. F major, C minor)
    for i, p in enumerate(piece.parts):
        part_midi = []
        time = []
        for n in p.notesAndRests.secondsMap: # secondsMap would include not only the note, but also the time information
            start = n['offsetSeconds']
            end = n['endTimeSeconds']
            time.append((start, end))
            element = n['element']
            try:
                part_midi.append(element.pitch.midi)
            except:
                try:
                    part_midi.append([item.midi for item in element.pitches])
                except:
                    part_midi.append(128) # coding rest as 128
        midi_dict[i] = part_midi
        time_dict[i] = time
    return key, midi_dict, time_dict

In [10]:
key, midi_dict, time_dict = quantify_music(sBach)

# Extract the whole corpus

## Get all the midi file name under this directory (including subdirectory)

In [6]:
import os
from glob import glob
midis = []
start_dir = "MIDI_Archive/Classical_Greatest"
pattern   = "*.mid"
for dir,_,_ in os.walk(start_dir):
    midis.extend(glob(os.path.join(dir,pattern))) 

In [7]:
len(midis)

3912

In [8]:
import json
midi_corpus = {}
count = 0
ind = 0
failed_midi = []
for midi in midis:
    try:
        piece = converter.parse(midi)
        key, midi_dict, duration_dict = quantify_music(piece)
        midi_corpus[midi.split(start_dir)[1].split('.mid')[0]] = [key, midi_dict, duration_dict]
        count+=1
    except:
        failed_midi.append(midi)
    if count > 200:
        json.dump(midi_corpus, open("midi_corpus_%s.json"%ind,'w'),indent = 4)
        ind +=1 
        count = 0
        break # I just extract the first 200 for now

yes
